# Tensorflow MLP
Here we will attempt to design a regression model for noisey sin data in tensorflow.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
sns.set()

seed_val = 2000
np.random.seed(seed_val)

Generate the data and add noise to it.

In [ ]:
def sin_data(noise_var=0.1, shuffle=False, scale=False, timestep=0.001, pi_factor=2):
    x = np.arange(0, pi_factor*np.pi, timestep)
    targets = np.sin(x)

#_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    noise = np.random.normal(0,noise_var,len(x))
    noisey_input = noise + x

    noise = np.random.normal(0,noise_var,len(targets))
    noisey_targets = noise + targets

    trainset = pd.DataFrame(columns=["NoiseyX", "NoiseyY"])
    trainset["NoiseyX"] = noisey_input
    trainset["NoiseyY"] = noisey_targets

    #_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    noise = np.random.normal(0,noise_var,len(x))
    noisey_input = noise + x

    noise = np.random.normal(0,noise_var,len(targets))
    noisey_targets = noise + targets

    testset = pd.DataFrame(columns=["NoiseyX", "NoiseyY"])
    testset["NoiseyX"] = noisey_input
    testset["NoiseyY"] = noisey_targets

    if scale:
       trainset['NoiseyX'] = MaxAbsScaler().fit_transform(trainset['NoiseyX'].values.reshape(-1,1))
       trainset['NoiseyY'] = MaxAbsScaler().fit_transform(trainset['NoiseyY'].values.reshape(-1,1))
       testset['NoiseyX'] = MaxAbsScaler().fit_transform(testset['NoiseyX'].values.reshape(-1,1))
       testset['NoiseyY'] = MaxAbsScaler().fit_transform(testset['NoiseyY'].values.reshape(-1,1))
    #_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    plt.scatter(x, targets, label="non-noisey sin", alpha=0.3)
    plt.scatter(trainset["NoiseyX"], trainset["NoiseyY"], label="noisey sin", marker='.')
    plt.scatter(testset["NoiseyX"], testset["NoiseyY"], label="test sin", marker='.')
    plt.legend()

    if shuffle:
        return trainset.sample(frac=1), testset #testset.sample(frac=1)
    else:
        return trainset, testset


In [ ]:
def linear_data(noise_var=0.1, shuffle=False, scale=False): 
    timestep = 0.1
    x = np.arange(0, 2*np.pi, timestep)
    targets = x

#_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    noise = np.random.normal(0,noise_var,len(x))
    noisey_input = noise + x

    noise = np.random.normal(0,noise_var,len(targets))
    noisey_targets = noise + targets

    trainset = pd.DataFrame(columns=["NoiseyX", "NoiseyY"])
    trainset["NoiseyX"] = noisey_input
    trainset["NoiseyY"] = noisey_targets

    #_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    noise = np.random.normal(0,noise_var,len(x))
    noisey_input = noise + x

    noise = np.random.normal(0,noise_var,len(targets))
    noisey_targets = noise + targets

    testset = pd.DataFrame(columns=["NoiseyX", "NoiseyY"])
    testset["NoiseyX"] = noisey_input
    testset["NoiseyY"] = noisey_targets

    #_______________________________________________________________________________________________________________________________________________________________________________________________________________________
    plt.scatter(x, targets, label="non-noisey sin", marker='.', alpha=0.3)
    plt.scatter(trainset["NoiseyX"], trainset["NoiseyY"], label="noisey train", marker='.')
    plt.scatter(testset["NoiseyX"], testset["NoiseyY"], label="noisey test", marker='.')
    plt.legend()

    if shuffle:
        return trainset.sample(frac=1), testset.sample(frac=1)
    else:
        return trainset, testset

In [ ]:
def add_bias(data):
    _N1 = np.shape(data)[0]
    _N2 = np.shape(data)[1]
    a = -1*np.ones((_N1,_N2+1))
    a[:,:-1] = data
    return a

Shuffle and split the noisey data into 60% train, 20% test and 20% valid.

In [ ]:
trainset, testset = sin_data(0.1, shuffle=True, scale=False, pi_factor=4)

print("training set length", len(trainset))

train_feat_no_bias = np.reshape(trainset["NoiseyX"].to_numpy(), (-1,1))
# train_feat = add_bias(train_feat_no_bias)
train_labels = np.reshape(trainset["NoiseyY"].to_numpy(), (-1,1))

test_feat_no_bias = np.reshape(testset["NoiseyX"].to_numpy(), (-1,1))
# test_feat = add_bias(test_feat_no_bias)
test_labels = np.reshape(testset["NoiseyY"].to_numpy(), (-1,1))

Here, will try tf to learn the data. 
### Note: by default, Kreas uses a bias in every layer so no need to add. 

In [ ]:
out_nodes = 1
in_features = 1

X = train_feat_no_bias.copy()
Y = train_labels.copy()

In [ ]:
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.06,
#     decay_steps=10000,
#     decay_rate=0.9)

In [ ]:
# define the mlp model
model = Sequential()
# add the hidden layers and non-linear activation functions
# model.add(Dense(10, input_shape=(in_features,), activation="sigmoid", kernel_initializer='ones', kernel_regularizer=keras.regularizers.l1(0.01)))
# model.add(Dense(100, input_shape=(in_features,), activation="relu"))
# model.add(Dense(40, input_shape=(in_features,), activation="relu"))
# model.add(Dense(100, input_shape=(in_features,), activation="relu"))
model.add(Dense(20, activation="tanh", use_bias=True))
model.add(Dense(20, activation="tanh", use_bias=True))
# add the output layer
model.add(Dense(out_nodes, activation="tanh", use_bias=True))

# optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss="mean_absolute_error", optimizer='SGD', metrics=["MSE","MAE"])
# model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(0.001), metrics=["MSE"])
history = model.fit(X, Y, epochs=100, batch_size=100, validation_split=0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.plot(hist['epoch'], hist['MAE'], label='Train Error')
plt.plot(hist['epoch'], hist['val_MAE'], label='Val Error')
plt.legend()

In [ ]:
hist.tail()

In [ ]:
test_input = test_feat_no_bias
test_output = test_labels

# test_input = train_feat_no_bias
# test_output = train_labels

prediction = model.predict(test_input)
accuracy = prediction-test_output

In [ ]:
if np.shape(test_input)[1] == 2:
    test_input = test_input[:,0]

plt.plot(test_input, test_output, label="TestData")
plt.plot(test_input, prediction, label="Prediction")
plt.plot(test_input,accuracy, label="Accuracy", alpha=0.4)
plt.legend()